In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 设置显示数字的千位分隔符
pd.set_option('display.float_format', '{:,.2f}'.format)

# 创建输出目录
output_dir = 'dengue_analysis_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取数据
df = pd.read_csv('processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
# 1. 按地区分组的深入分析
print("1. 按地区分析病例分布")

# 按地区和年份分组统计病例和死亡
region_yearly = df.groupby(['Region', 'Year']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 添加计算死亡率列
region_yearly['Mortality_Rate'] = (region_yearly['Dengue_Deaths'] / region_yearly['Dengue_Cases']) * 100

# 查看前几行
print("按地区和年份统计（前5行）：")
print(region_yearly.head())

# 计算各地区总体情况
region_total = df.groupby('Region').agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 添加计算占比列
region_total['Cases_Percentage'] = region_total['Dengue_Cases'] / region_total['Dengue_Cases'].sum() * 100
region_total['Deaths_Percentage'] = region_total['Dengue_Deaths'] / region_total['Dengue_Deaths'].sum() * 100
region_total['Mortality_Rate'] = (region_total['Dengue_Deaths'] / region_total['Dengue_Cases']) * 100

# 按病例数降序排序
region_total = region_total.sort_values('Dengue_Cases', ascending=False)

print("\n各地区总体情况（按病例数降序）：")
print(region_total.to_string())

# 保存为CSV文件
region_total.to_csv(f'{output_dir}/region_total_summary.csv', index=False)

1. 按地区分析病例分布
按地区和年份统计（前5行）：
  Region  Year  Dengue_Cases  Dengue_Deaths  Mortality_Rate
0  BARMM  2016          2191            243           11.09
1  BARMM  2017           485              2            0.41
2  BARMM  2018          2460             33            1.34
3  BARMM  2019          5393             31            0.57
4  BARMM  2020          1008             23            2.28

各地区总体情况（按病例数降序）：
         Region  Dengue_Cases  Dengue_Deaths  Cases_Percentage  Deaths_Percentage  Mortality_Rate
6   Region IV-A        163029            652             14.21               3.87            0.40
5    Region III        131064            482             11.42               2.86            0.37
10    Region VI        117523           1825             10.24              10.83            1.55
2           NCR        115966           4008             10.11              23.79            3.46
11   Region VII        110683           1760              9.65              10.45            1.59
13    

In [6]:
# 2. 计算各地区关键统计量
print("\n2. 计算各地区关键统计量")

# 使用agg方法计算多种统计量
region_stats = df.groupby('Region').agg({
    'Dengue_Cases': ['mean', 'median', 'std', 'min', 'max', 'count'],
    'Dengue_Deaths': ['mean', 'median', 'std', 'min', 'max', 'sum']
})

# 按平均病例数降序排序
region_stats = region_stats.sort_values(('Dengue_Cases', 'mean'), ascending=False)

print("各地区关键统计量（前5个地区）：")
print(region_stats.head().to_string())

# 保存为CSV文件
region_stats.to_csv(f'{output_dir}/region_statistics.csv')



2. 计算各地区关键统计量
各地区关键统计量（前5个地区）：
            Dengue_Cases                                     Dengue_Deaths                              
                    mean   median      std  min    max count          mean median    std min   max   sum
Region                                                                                                  
Region IV-A     2,717.15 2,096.00 3,485.74   77  21658    60         10.87   8.00  10.42   0    48   652
Region III      2,184.40 1,834.50 1,761.88  201   9713    60          8.03   5.00  15.65   0   121   482
Region VI       1,958.72   783.00 3,135.24  115  18090    60         30.42   5.00 118.01   0   668  1825
NCR             1,932.77 1,276.00 1,936.96   57  11033    60         66.80   7.50 267.15   0  1651  4008
Region VII      1,844.72 1,227.00 1,451.21  207   5534    60         29.33   9.00  96.15   0   584  1760


In [11]:
# 1. 按地区分组的深入分析
print("1. 按地区分析病例分布")

# 按地区和年份分组统计病例和死亡
region_yearly = df.groupby(['Region', 'Year']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 添加计算死亡率列
region_yearly['Mortality_Rate'] = (region_yearly['Dengue_Deaths'] / region_yearly['Dengue_Cases']) * 100

# 计算各地区总体情况
region_total = df.groupby('Region').agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 添加计算占比列
region_total['Cases_Percentage'] = region_total['Dengue_Cases'] / region_total['Dengue_Cases'].sum() * 100
region_total['Deaths_Percentage'] = region_total['Dengue_Deaths'] / region_total['Dengue_Deaths'].sum() * 100
region_total['Mortality_Rate'] = (region_total['Dengue_Deaths'] / region_total['Dengue_Cases']) * 100

# 按病例数降序排序
region_total = region_total.sort_values('Dengue_Cases', ascending=False)

# 保存为CSV文件
region_total.to_csv(f'{output_dir}/region_total_summary.csv', index=False)

# 可视化：全国时间序列趋势图
national_timeseries = df.groupby(['Year', 'Month', 'MonthNum', 'Date']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()

# 按日期排序
national_timeseries = national_timeseries.sort_values('Date')

plt.figure(figsize=(14, 7))
plt.plot(national_timeseries['Date'], national_timeseries['Dengue_Cases'], 
         marker='o', linestyle='-', markersize=4, linewidth=2)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾全国登革热病例时间序列趋势（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
else:
    # 回退到默认方法
    plt.title('菲律宾全国登革热病例时间序列趋势（2016-2020）', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('病例数', fontsize=12)

plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# 添加主要年份标记
plt.axvline(pd.to_datetime('2017-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2018-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2019-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2020-01-01'), color='gray', linestyle='--', alpha=0.7)

# 保存图表
plt.savefig(f'{output_dir}/national_dengue_trend.png', dpi=300)
print(f"保存图表：{output_dir}/national_dengue_trend.png")
plt.close()

# 可视化：绘制各地区年度病例趋势图
top5_regions = region_total.head(5)['Region'].tolist()
plt.figure(figsize=(14, 8))

for region in top5_regions:
    # 获取该地区的年度数据
    region_data = region_yearly[region_yearly['Region'] == region]
    plt.plot(region_data['Year'], region_data['Dengue_Cases'], marker='o', linewidth=2, label=region)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾前5个高发地区年度登革热病例趋势（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('年份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(title='地区', prop=chinese_font)
else:
    # 回退到默认方法
    plt.title('菲律宾前5个高发地区年度登革热病例趋势（2016-2020）', fontsize=16)
    plt.xlabel('年份', fontsize=12)
    plt.ylabel('病例数', fontsize=12)
    plt.legend(title='地区')

plt.grid(True)
plt.xticks(range(2016, 2021))
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/top5_regions_yearly_trend.png', dpi=300)
print(f"保存图表：{output_dir}/top5_regions_yearly_trend.png")
plt.close()

1. 按地区分析病例分布
保存图表：dengue_analysis_results/national_dengue_trend.png


C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_13680\4041793513.py:93: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_13680\4041793513.py:93: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_13680\4041793513.py:96: UserWarning: Glyph 22320 (\N{CJK UNIFIED IDEOGRAPH-5730}) missing from font(s) Arial.
  plt.savefig(f'{output_dir}/top5_regions_yearly_trend.png', dpi=300)
C:\Users\nnnnghj\AppData\Local\Temp\ipykernel_13680\4041793513.py:96: UserWarning: Glyph 21306 (\N{CJK UNIFIED IDEOGRAPH-533A}) missing from font(s) Arial.
  plt.savefig(f'{output_dir}/top5_regions_yearly_trend.png', dpi=300)


保存图表：dengue_analysis_results/top5_regions_yearly_trend.png


In [14]:
# 2. 季节性模式分析
print("\n2. 季节性模式分析")

# 按地区和月份分组计算平均病例数
monthly_by_region = df.groupby(['Region', 'Month', 'MonthNum']).agg({
    'Dengue_Cases': 'mean',
    'Dengue_Deaths': 'mean'
}).reset_index()

# 确保月份按正确顺序排序
monthly_by_region = monthly_by_region.sort_values(['Region', 'MonthNum'])

# 找出各地区的高峰月份
peak_months = monthly_by_region.loc[monthly_by_region.groupby('Region')['Dengue_Cases'].idxmax()]
peak_months = peak_months.sort_values('Dengue_Cases', ascending=False)

# 保存高峰月份数据
peak_months.to_csv(f'{output_dir}/region_peak_months.csv', index=False)

# 可视化：季节性模式分析 - 箱线图
plt.figure(figsize=(15, 8))
sns.boxplot(x='MonthNum', y='Dengue_Cases', data=df)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾登革热病例季节性分布（箱线图）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('病例数', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                 '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(12), [name for name in month_names], fontproperties=chinese_font)
else:
    # 回退到默认方法
    plt.title('菲律宾登革热病例季节性分布（箱线图）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('病例数', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                 '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(12), month_names)

plt.grid(True, axis='y')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/seasonal_boxplot.png', dpi=300)
print(f"保存图表：{output_dir}/seasonal_boxplot.png")
plt.close()

# 可视化：按月份创建柱状图
monthly_avg = df.groupby(['MonthNum', 'Month'])['Dengue_Cases'].mean().reset_index()
monthly_avg = monthly_avg.sort_values('MonthNum')

plt.figure(figsize=(12, 6))
ax = sns.barplot(x='MonthNum', y='Dengue_Cases', data=monthly_avg)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾登革热病例季节性模式（2016-2020平均值）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('平均病例数', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(12), [name for name in month_names], fontproperties=chinese_font)
else:
    # 回退到默认方法
    plt.title('菲律宾登革热病例季节性模式（2016-2020平均值）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('平均病例数', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(12), month_names)

plt.grid(True, axis='y')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/seasonal_barplot.png', dpi=300)
print(f"保存图表：{output_dir}/seasonal_barplot.png")
plt.close()


2. 季节性模式分析
保存图表：dengue_analysis_results/seasonal_boxplot.png
保存图表：dengue_analysis_results/seasonal_barplot.png


In [15]:
# 1. 创建地区-月份热图
print("1. 创建地区-月份热图")

# 计算各地区各月份的平均病例数
heatmap_data = df.groupby(['Region', 'MonthNum']).agg({
    'Dengue_Cases': 'mean'
}).reset_index()

# 创建枢轴表用于热图
heatmap_pivot = heatmap_data.pivot(index='Region', columns='MonthNum', values='Dengue_Cases')

# 按总病例数降序排序地区
region_order = region_total['Region'].tolist()
heatmap_pivot = heatmap_pivot.reindex(region_order)

# 绘制热图
plt.figure(figsize=(14, 10))
ax = sns.heatmap(heatmap_pivot, cmap='YlOrRd', annot=True, fmt='.0f', linewidths=0.5)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('各地区登革热病例月度分布热图（2016-2020平均值）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('地区', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    ax.set_xticklabels([name for name in month_names], fontproperties=chinese_font)
else:
    # 回退到默认方法
    plt.title('各地区登革热病例月度分布热图（2016-2020平均值）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('地区', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    ax.set_xticklabels(month_names)

plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/region_month_heatmap.png', dpi=300)
print(f"保存图表：{output_dir}/region_month_heatmap.png")
plt.close()


1. 创建地区-月份热图
保存图表：dengue_analysis_results/region_month_heatmap.png


In [16]:
# 2. 按年度和月份创建热图
yearly_monthly_data = df.groupby(['Year', 'MonthNum']).agg({
    'Dengue_Cases': 'sum'
}).reset_index()

# 创建枢轴表
yearly_heatmap = yearly_monthly_data.pivot(index='Year', columns='MonthNum', values='Dengue_Cases')

# 绘制热图
plt.figure(figsize=(12, 6))
ax = sns.heatmap(yearly_heatmap, cmap='YlOrRd', annot=True, fmt='.0f', linewidths=0.5)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('登革热病例年度和月度分布热图（全国）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('年份', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    ax.set_xticklabels([name for name in month_names], fontproperties=chinese_font)
else:
    # 回退到默认方法
    plt.title('登革热病例年度和月度分布热图（全国）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('年份', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    ax.set_xticklabels(month_names)

plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/yearly_monthly_heatmap.png', dpi=300)
print(f"保存图表：{output_dir}/yearly_monthly_heatmap.png")
plt.close()

保存图表：dengue_analysis_results/yearly_monthly_heatmap.png


In [17]:
# 3. 死亡病例与死亡率分析
print("\n2. 死亡病例与死亡率分析")

# 添加计算死亡率列
df['Mortality_Rate'] = np.where(df['Dengue_Cases'] > 0, 
                                (df['Dengue_Deaths'] / df['Dengue_Cases']) * 100, 
                                0)  # 避免除以零

# 按年度计算死亡率
yearly_mortality = df.groupby('Year').agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()
yearly_mortality['Mortality_Rate'] = (yearly_mortality['Dengue_Deaths'] / yearly_mortality['Dengue_Cases']) * 100

# 按月份计算平均死亡率
monthly_mortality = df.groupby(['Month', 'MonthNum']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()
monthly_mortality['Mortality_Rate'] = (monthly_mortality['Dengue_Deaths'] / monthly_mortality['Dengue_Cases']) * 100
monthly_mortality = monthly_mortality.sort_values('MonthNum')

# 保存死亡率分析结果
yearly_mortality.to_csv(f'{output_dir}/yearly_mortality.csv', index=False)
monthly_mortality.to_csv(f'{output_dir}/monthly_mortality.csv', index=False)

# 可视化：年度死亡率变化
plt.figure(figsize=(10, 6))
plt.bar(yearly_mortality['Year'], yearly_mortality['Mortality_Rate'], color='darkred')

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾登革热年度死亡率变化（2016-2020）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('年份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('死亡率（%）', fontproperties=chinese_font, fontsize=12)
else:
    # 回退到默认方法
    plt.title('菲律宾登革热年度死亡率变化（2016-2020）', fontsize=16)
    plt.xlabel('年份', fontsize=12)
    plt.ylabel('死亡率（%）', fontsize=12)

plt.xticks(yearly_mortality['Year'])
plt.grid(axis='y')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/yearly_mortality_rate.png', dpi=300)
print(f"保存图表：{output_dir}/yearly_mortality_rate.png")
plt.close()

# 可视化：月度死亡率变化
plt.figure(figsize=(12, 6))
plt.bar(monthly_mortality['MonthNum'], monthly_mortality['Mortality_Rate'], color='darkred')

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾登革热月度死亡率（2016-2020平均）', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('死亡率（%）', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(1, 13), [name for name in month_names], fontproperties=chinese_font)
else:
    # 回退到默认方法
    plt.title('菲律宾登革热月度死亡率（2016-2020平均）', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('死亡率（%）', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(1, 13), month_names)

plt.grid(axis='y')
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/monthly_mortality_rate.png', dpi=300)
print(f"保存图表：{output_dir}/monthly_mortality_rate.png")
plt.close()



2. 死亡病例与死亡率分析
保存图表：dengue_analysis_results/yearly_mortality_rate.png
保存图表：dengue_analysis_results/monthly_mortality_rate.png


In [18]:
# 3. 病例数与死亡率关系的散点图
print("\n3. 高级可视化与关联分析")

plt.figure(figsize=(12, 8))
sns.scatterplot(x='Dengue_Cases', y='Mortality_Rate', 
                hue='Region', size='Dengue_Deaths',
                sizes=(20, 200), alpha=0.7, data=df)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('登革热病例数与死亡率关系散点图', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('病例数', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('死亡率（%）', fontproperties=chinese_font, fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', prop=chinese_font)
else:
    # 回退到默认方法
    plt.title('登革热病例数与死亡率关系散点图', fontsize=16)
    plt.xlabel('病例数', fontsize=12)
    plt.ylabel('死亡率（%）', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/cases_vs_mortality_scatter.png', dpi=300)
print(f"保存图表：{output_dir}/cases_vs_mortality_scatter.png")
plt.close()

# 创建时间序列图：显示病例数和死亡率的变化
# 按月份汇总全国数据
monthly_timeseries = df.groupby(['Year', 'MonthNum', 'Date']).agg({
    'Dengue_Cases': 'sum',
    'Dengue_Deaths': 'sum'
}).reset_index()
monthly_timeseries['Mortality_Rate'] = (monthly_timeseries['Dengue_Deaths'] / monthly_timeseries['Dengue_Cases']) * 100
monthly_timeseries = monthly_timeseries.sort_values('Date')

# 绘制双Y轴图表
fig, ax1 = plt.subplots(figsize=(14, 8))

# 左Y轴：病例数
color = 'tab:blue'
ax1.set_xlabel('日期', fontsize=12)
ax1.set_ylabel('病例数', color=color, fontsize=12)
ax1.plot(monthly_timeseries['Date'], monthly_timeseries['Dengue_Cases'], color=color, marker='o', linestyle='-', alpha=0.7)
ax1.tick_params(axis='y', labelcolor=color)

# 右Y轴：死亡率
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('死亡率（%）', color=color, fontsize=12)
ax2.plot(monthly_timeseries['Date'], monthly_timeseries['Mortality_Rate'], color=color, marker='x', linestyle='-', alpha=0.7)
ax2.tick_params(axis='y', labelcolor=color)

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('菲律宾登革热病例数和死亡率时间序列（2016-2020）', fontproperties=chinese_font, fontsize=16)
    ax1.set_xlabel('日期', fontproperties=chinese_font, fontsize=12)
    ax1.set_ylabel('病例数', color='tab:blue', fontproperties=chinese_font, fontsize=12)
    ax2.set_ylabel('死亡率（%）', color='tab:red', fontproperties=chinese_font, fontsize=12)
else:
    # 回退到默认方法
    plt.title('菲律宾登革热病例数和死亡率时间序列（2016-2020）', fontsize=16)

# 添加主要年份标记
plt.axvline(pd.to_datetime('2017-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2018-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2019-01-01'), color='gray', linestyle='--', alpha=0.7)
plt.axvline(pd.to_datetime('2020-01-01'), color='gray', linestyle='--', alpha=0.7)

ax1.grid(True)
fig.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/cases_and_mortality_timeseries.png', dpi=300)
print(f"保存图表：{output_dir}/cases_and_mortality_timeseries.png")
plt.close()


3. 高级可视化与关联分析
保存图表：dengue_analysis_results/cases_vs_mortality_scatter.png
保存图表：dengue_analysis_results/cases_and_mortality_timeseries.png


In [ ]:
# 4. 2020年与之前年份的对比分析（可能受COVID-19影响）
# 绘制2020年与之前年份的月度对比图
plt.figure(figsize=(14, 8))

# 计算2016-2019年每月平均值
pre_2020 = df[df['Year'] < 2020].groupby('MonthNum')['Dengue_Cases'].mean()
year_2020 = df[df['Year'] == 2020].groupby('MonthNum')['Dengue_Cases'].mean()

plt.plot(pre_2020.index, pre_2020.values, marker='o', linewidth=2, label='2016-2019平均')
plt.plot(year_2020.index, year_2020.values, marker='x', linewidth=2, label='2020年')

# 使用字体对象显式设置所有文本元素
if chinese_font is not None:
    plt.title('2020年与2016-2019年平均登革热病例数月度对比', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('月份', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('平均病例数', fontproperties=chinese_font, fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(1, 13), [name for name in month_names], fontproperties=chinese_font)
    plt.legend(prop=chinese_font)
else:
    # 回退到默认方法
    plt.title('2020年与2016-2019年平均登革热病例数月度对比', fontsize=16)
    plt.xlabel('月份', fontsize=12)
    plt.ylabel('平均病例数', fontsize=12)
    month_names = ['一月', '二月', '三月', '四月', '五月', '六月', 
                  '七月', '八月', '九月', '十月', '十一月', '十二月']
    plt.xticks(range(1, 13), month_names)
    plt.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/2020_vs_previous_years.png', dpi=300)
print(f"保存图表：{output_dir}/2020_vs_previous_years.png")
plt.close()

保存图表：dengue_analysis_results/2020_vs_previous_years.png
